In [ ]:
from selenium import webdriver #another option is to import urllib.request if selenium is not available
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup #Use bs4 by installing beautifulsoup4 in Python3
import pandas as pd

In [ ]:
driver = webdriver.Chrome("/usr/local/bin/chromedriver") #Make sure chromedriver is installed beforehand

In [ ]:
pageUrl = []
begUrl = "https://community.microstrategy.com/s/global-search/%40uri?language=en_US#"
endUrl = "&numberOfResults=100&f:Facet_Topics=[All,Clients,Web]&f:Facet_commoncontenttype=[Knowledge%20Base]&f:Facet_commonLanguage=[English]"

for i in range(0,10):
    if i == 0:
        Url = "https://community.microstrategy.com/s/global-search/%40uri?language=en_US#sort=relevancy&numberOfResults=100&f:Facet_Topics=[All,Clients,Web]&f:Facet_commoncontenttype=[Knowledge%20Base]&f:Facet_commonLanguage=[English]"
    else:
        pageNum = i*100
        Url = begUrl + "first=" + str(pageNum) + "&sort=relevancy" + endUrl
    
    pageUrl.append(Url)

    if i == 4 or 9:
        dfP = pd.DataFrame({"Page_URL":pageUrl})
        dfP.to_csv('Page_URL.csv', index=False, encoding='utf-8')

print(pageUrl)

In [ ]:
kbLinks = [] #create empty list to append from for loop later
for p in pageUrl:
    driver.get(p)


    delay = 10 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.CoveoCustomSalesforceCommunityResultLink"))) #Make sure ALL instances of "CoveoCustomSalesforceCommunityResultLink" element are found instead of just first one before closing webpage 
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")

    content = driver.page_source
    soup = BeautifulSoup(content)

    for link in soup.find_all('a',class_='CoveoCustomSalesforceCommunityResultLink CoveoResultLink',href=True):
        kbLinks.append(link.get('href')) #copies the URL links
    
    if pageUrl.index(p) == 3 or 5 or 7 or 9:
        dfK = pd.DataFrame({"Link":kbLinks})
        dfK.to_csv('KB_Links.csv', index=False, encoding='utf-8')   



In [ ]:
authors = []
link = []
linkUnloaded = []
for l in kbLinks:
    driver.get(l)

    delay = 10 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "span.userPhoto"))) #"span.userInfo username" takes too much time so used "userPhoto" instead

        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
    contentB = driver.page_source
    soupB = BeautifulSoup(contentB)

    for a in soupB.find_all('span',class_='userInfo userName'):
        if a.text != '':
            authors.append(a.text)
            link.append(l)
        else:
            linkUnloaded.append(l) #LS Note: prints nothing at this time, not sure how to fix

    if kbLinks.index(l) == 199 or 399 or 599 or 799 or 999: #LS Note: This if statement still prints for every index where instead I wanted it to print only for specific indices.  I'm not sure how to fix.
        dfA = pd.DataFrame({'Author':authors,'Link':link})
        dfA.to_csv('authors.csv', index=False, encoding='utf-8')
        pct = round((kbLinks.index(l)/999)*100,0)
        print('\n----------\n' + str(pct) +  ' percent progress\n----------\n')
  
a = {'Author':authors,'Link':link}
b = {'Unloaded Link':linkUnloaded}
dfA = pd.DataFrame(a)
dfA.to_csv('authors.csv', index=False, encoding='utf-8')

dfB = pd.DataFrame(b)
dfB.to_csv('Unloaded Links.csv', index=False, encoding='utf-8')

print("Loading done! You can close browser.")